# Modelado de Series Temporales de Aparcamientos usando RNNs y optimización con Optuna
En este notebook se abordará un problema de predicción de series temporales utilizando datos de disponibilidad de parkings.

Flujo de trabajo:
1. Agrupación de los datos por id de aparcamiento (idAparcamiento), para tratar cada parking como una serie temporal independiente.
2. División del conjunto de datos en tres subconjuntos: entrenamiento (train), validación (val) y prueba (test).
3. Entrenamiento de modelos de redes neuronales recurrentes simples (vanilla):
 - Vanilla RNN
 - Vanilla GRU
 - Vanilla LSTM
4. Ajuste de hiperparámetros utilizando Optuna para encontrar la configuración óptima en cada tipo de modelo.
5. Comparación de los resultados de rendimiento entre los distintos modelos utilizando métricas apropiadas.

El objetivo final es evaluar qué arquitectura ofrece mejores resultados para este tipo de datos y tarea de predicción.


In [9]:
import pandas as pd
import numpy as np
import torch
import optuna


In [10]:
df = pd.read_csv("../data/processed/data_processed.csv")

#convertir a indicr
df.set_index("timestamp", inplace= True)
df.index = pd.to_datetime(df.index)

df

,idAparcamiento,PlazasTotales,PlazasDisponibles,PorcPlazasDisponibles,year,month,day,weekday
timestamp,,,,,,,,
2023-02-03 10:00:00,6,372.0,60.0,16.129032,2023,2,3,4
2023-02-03 11:00:00,6,372.0,48.0,12.903226,2023,2,3,4
2023-02-03 12:00:00,6,372.0,66.0,17.741935,2023,2,3,4
2023-02-03 13:00:00,6,372.0,119.0,31.989247,2023,2,3,4
2023-02-03 14:00:00,6,372.0,155.0,41.666667,2023,2,3,4
...,...,...,...,...,...,...,...,...
2025-03-05 03:00:00,78,464.0,355.0,76.508621,2025,3,5,2
2025-03-05 04:00:00,78,464.0,355.0,76.508621,2025,3,5,2
2025-03-05 05:00:00,78,464.0,356.0,76.724138,2025,3,5,2


### División del conjunto de datos

Realizamos la división del conjunto de datos, agrupando por `idAparcamiento`. 
La idea es que el conjunto de **test sea común** para todos los aparcamientos, correspondiente al **último 10% del rango temporal total** del dataset. 

El resto de los datos disponibles para cada parking se dividen en:

- **Entrenamiento (train)**: el primer 85% de los datos previos al test.
- **Validación (val)**: el último 15% de los datos previos al test.

In [21]:
from datetime import timedelta

# 1. Calcular el rango temporal global
fecha_min_global = df.index.min()
fecha_max_global = df.index.max()
rango_total = fecha_max_global - fecha_min_global

# 2. Calcular el inicio del conjunto de test (último 10% del rango)
test_ratio = 0.10
test_duration = timedelta(seconds=rango_total.total_seconds() * test_ratio)
test_start = fecha_max_global - test_duration

# 3. Diccionarios para almacenar los splits
train_dict = {}
val_dict = {}
test_dict = {}

val_ratio = 0.15  # del conjunto anterior al test

# 4. División por parking
for parking_id, group in df.groupby("idAparcamiento"):
    group = group.sort_index()
    
    # Split basado en el corte global
    test_set = group[group.index >= test_start]
    remaining = group[group.index < test_start]

    if len(remaining) < 10:
        continue  # O manejar parkings con poca data antes del test
    
    # Dividir el resto en train y val
    val_size = int(len(remaining) * val_ratio)
    val_set = remaining.iloc[-val_size:]
    train_set = remaining.iloc[:-val_size]
    
    # Guardar resultados
    train_dict[parking_id] = train_set
    val_dict[parking_id] = val_set
    test_dict[parking_id] = test_set




## Definir Dataset de pytorch

### Definir modelos vanilla